In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium_stealth import stealth

import time
import random
import string
import json
import pafy
from scipy import stats
from collections import defaultdict

import pandas as pd
import numpy as np

# Hide video unavailable warnings
import warnings
warnings.filterwarnings("ignore")

# Data Inladen

In [2]:
mails = pd.read_csv("../data/mails.csv")

In [3]:
videos = pd.read_csv("../data/uploads.csv")

# Helpers

In [4]:
def vid_to_watch(videos, usertype = 1):
    """Finds a conspiracy video to watch and 
    makes sure it's still accessible and under 2 hours long"""
    
    if usertype == 1:
        # Choose a non-conspiracy video to watch
        to_watch = np.random.choice(videos[~videos["conspiracy"]]["video_id"])
    else:
        # Choose a conspiracy video to watch
        to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
    
    # Check its length
    url = f"http://www.youtube.com/watch?v={to_watch}"
    try:
        video = pafy.new(url);
        vid_len = video.length
    except: # Video no longer accessible
        vid_len = 9999
    
    # Videos over 2 hours will be skipped
    while vid_len > 7200:
        to_watch = np.random.choice(videos[videos["conspiracy"]]["video_id"])
        
        url = f"http://www.youtube.com/watch?v={to_watch}"
        try:
            video = pafy.new(url);
            vid_len = video.length
        except:
            vid_len = 9999
            
    # Calculate how much of the video will be watched
    percentage = np.random.normal(0.55, 0.25)
    
    # Make sure a video is watched at most 100%
    if percentage > 1:
        percentage = 1
    
    watch_time = percentage * vid_len

    return watch_time, to_watch

In [5]:
def init_driver():
    """Initializes a selenium driver in stealth mode"""
        
    # Add options
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-infobars")

    options.add_argument("start-maximized")

    # options.add_argument("--headless")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    
    # Start driver
    driver = webdriver.Chrome(options=options)

    # Hide the fact we're using a bot
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    
    return driver

In [6]:
def login_google(driver, mail, password):
    """Logs in on Google given a driver, email-address and password.
    This code only works if the email-address was made within chromedriver itself"""
    
    # login on google
    driver.get("https://accounts.google.com/ServiceLogin")
    time.sleep(np.random.uniform(1, 1.5))

    # Fillin mail
    driver.find_element_by_id("identifierId").send_keys(mail)
    driver.find_element_by_id("identifierNext").click()

    time.sleep(np.random.uniform(1, 1.5))

    # Fill in password
    driver.find_element_by_xpath("//input[@name = 'password']").send_keys(password)
    driver.find_element_by_xpath('//button[contains(@class, "VfPpkd-LgbsSe")]').click()

    time.sleep(0.5)

    # If security-check is asked, skip it
    try:
        driver.find_element_by_xpath("//div[contains(@class, 'U26fgb O0WRkf')]").click()
    except:
        pass

In [7]:
def in_seconds(timestamp):
    """Converts a YouTube timestamp to a number of seconds"""
    # Order times in ascending order
    times = timestamp.split(":")[::-1]
    
    # Convert to seconds
    return sum([int(times[i]) * 60**i for i in range(len(times))])

In [81]:
def prepare_video(driver):
    """Does everything necessary to start watching a video"""
    
    time.sleep(1)
    
    # Some videos get a warning for inappropriate content
    content_warning = driver.find_elements_by_xpath(
        "//paper-button[text() = 'I understand and wish to proceed']")
    
    if content_warning:
        content_warning[0].click()

    # Skip ad(s) if they are there
    for ad in range(2):
        time.sleep(0.5)
        if driver.find_elements_by_xpath("//img[@class = 'ytp-ad-image']"):
            time.sleep(6)
            try:
                driver.find_element_by_xpath("//div[@class = 'ytp-ad-text ytp-ad-skip-button-text']").click()
            except:
                pass

    # Toggle autoplayer
    time.sleep(0.5)
    autoplay = driver.find_element_by_xpath("//div[@class = 'ytp-autonav-toggle-button']")
    
    if autoplay.get_attribute("aria-checked") == "true":
        autoplay.click()
        
    pause = driver.find_element_by_xpath("//button[contains(@class, 'ytp-play-button')]")
    
    if pause.get_attribute("aria-label") == "Play (k)":
        pause.click()

In [82]:
def check_video_running(driver, watch_time):
    timestamp = driver.find_element_by_class_name("ytp-time-current").text
            
    # Sometimes the timestamp cannot be displayed
    if timestamp:   
        return in_seconds(timestamp) < watch_time
                    
    # Check if the video is done (can happen when ads play and the video is very short)
    try:
        driver.find_element_by_xpath("//div[contains(@class, 'ended-mode')]")
        return False
    except:
        pass
    
    # Video is still running
    return True

In [83]:
def zipf(alpha, N):
    """Calculates Zipf distribution given variables"""
    denominator = sum([1/n**alpha for n in range(1, N + 1)])
    return [(1/k**alpha)/denominator for k in range(1, N + 1)]

In [84]:
def choose_recommendation(recommendations, usertype = 3):
    """Selects a video to watch based on a top 10 of given recommendations"""
 
    # Choose a video according to the zipf-distribution of video-selection (Zhou et al., 2010)    
    odds = zipf(0.78, 20)    
    choice = np.random.choice(recommendations, p = odds)   
        
    vid_len = pafy.new(choice).length
    
    # Make sure no livestreams are clicked
    while not vid_len or vid_len > 7200:
        choice = np.random.choice(recommendations, p = odds)
        vid_len = pafy.new(choice).length
            
    # Calculate how much of the video will be watched
    watch_time = np.random.normal(0.55, 0.25) * vid_len
    
    return watch_time, choice.replace("https://www.youtube.com/watch?v=", "")

In [85]:
def get_video_info(driver):
    """Gets basic information about a video"""
    
    views = driver.find_element_by_xpath("//span[contains(@class, 'view-count')]").text
    likes, dislikes = driver.find_elements_by_xpath("//yt-formatted-string[contains(@aria-label, 'likes')]")
    date = driver.find_element_by_xpath("//div[@id = 'date']/yt-formatted-string").text
    url = driver.current_url
    
    return int(views.replace(",", "").replace(" views", "")), likes.text, dislikes.text, date, url

# Het experiment

In [86]:
def run_experiment(videos, mails, usertype=1):
    experiment_results = defaultdict(list)
    videos_per_user = defaultdict(list)

    for user in mails.itertuples():
        # Find current user's info
        mail, password = user[1], user[2]

        # Initialize driver
        driver = init_driver()    

        # Login to google
        login_google(driver, mail, password)
        
        rec_to_watch = None

        # Start watching videos
        for i in range(1, 3):
            if not rec_to_watch:
                # Get a random video from the dataset and watch it
                watch_time, to_watch = vid_to_watch(videos, usertype)
                driver.get(f"http://youtube.com/watch?v={to_watch}")
                vid_running = True
            else:
                driver.get(f"http://youtube.com/watch?v={rec_to_watch}")
                vid_running = True

            # Skip ads, disable autoplay
            prepare_video(driver)
            
            # Save information of current video being watched
            views, likes, dislikes, date, url = get_video_info(driver)
            
            videos_per_user["user"].append(mail)
            videos_per_user["video_number"].append(i)
            videos_per_user["url"].append(url)
            videos_per_user["views"].append(views)
            videos_per_user["likes"].append(likes)
            videos_per_user["dislikes"].append(dislikes)
            videos_per_user["date"].append(date)           

            # Watch video
            while check_video_running(driver, watch_time):
                time.sleep(1)
            
            # If we have a usertype that relies on direct recommendations
            if usertype == 3:
                video_recs = driver.find_elements_by_xpath("//a[contains(@class, 'ytd-compact-video-renderer')]")
                watch_time, rec_to_watch = choose_recommendation([rec.get_attribute("href")
                                                                  for rec in video_recs][:20],
                                                                 usertype)

            # Go to the youtube homepage
            driver.get("http://youtube.com")
            time.sleep(1)

            # Get videos on youtube home
            channels = driver.find_elements_by_xpath("//a[@id = 'avatar-link']")
            vids = driver.find_elements_by_xpath("//a[@id = 'video-title-link']")

            # If we have a user that relies on homepage recommendations
            if usertype == 4:
                watch_time, rec_to_watch = choose_recommendation([rec.get_attribute("href")
                                                                  for rec in vids][:20],
                                                                 usertype)
            
            # Get top 20 recommendations
            for rec in range(20):
                experiment_results["user"].append(mail)
                experiment_results["vids_watched"].append(i)

                experiment_results["video"].append(vids[rec].get_attribute("href"))
                experiment_results["channel"].append(channels[rec].get_attribute("href"))
        break

    return pd.DataFrame(experiment_results), pd.DataFrame(videos_per_user)

df1, df2 = run_experiment(videos, mails, 3)

In [87]:
df1

,user,vids_watched,video,channel
0,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=rMw9sJ6e6Gc,https://www.youtube.com/c/BLCKBX
1,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=b3waibCUoKo,https://www.youtube.com/c/Selfsufficientme
2,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=BCAQ2DdKAXA,https://www.youtube.com/user/MrCaligula22
3,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=yaWVflQolmM,https://www.youtube.com/c/Richroll
4,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=POS6ftKhDAs,https://www.youtube.com/c/I24newsTv
5,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=JWJgFT24dXg,https://www.youtube.com/user/lisab1230
6,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=bDnA_coA168,https://www.youtube.com/channel/UCf6ucYmkyUzQs...
7,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=sLUMfjevre4,https://www.youtube.com/c/Weltverborgen
8,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=VH6msTsClUs,https://www.youtube.com/channel/UCdAeRRjq6vjnA...
9,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=KRXl_N0OtX4,https://www.youtube.com/c/TomBilyeu


In [88]:
df2

,user,video_number,url,views,likes,dislikes,date
0,scriptiebot@gmail.com,1,https://www.youtube.com/watch?v=3lCTFj5N3tg,1517,30,7,"Apr 28, 2012"
1,scriptiebot@gmail.com,2,https://www.youtube.com/watch?v=da1vvigy5tQ&t=...,6976278,105K,7.6K,"May 4, 2015"
